In [3]:
from glob import iglob
import os
from time import sleep

import ensembl_rest
from joblib import Parallel, delayed
import pandas as pd
import screed
import seaborn as sns
from tqdm import tqdm

In [4]:
cd /mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019

/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019


In [5]:
ls -lha

total 2.8G
drwxr-xr-x  5 olga root 4.0K Jan 17 08:01 ./
drwxr-xr-x  3 olga root 4.0K Dec 25 17:48 ../
drwxr-xr-x  5 olga czb  4.0K Dec 26 19:44 Archaea/
drwxr-xr-x  5 olga czb   16K Dec 26 19:44 Bacteria/
drwxr-xr-x 12 olga czb   32K Jan 17 08:01 Eukaryota/
-rw-r--r--  1 olga czb  754K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.csv
-rw-r--r--  1 olga czb   68K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.csv.gz
-rw-r--r--  1 olga czb  133K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.parquet
-rw-r--r--  1 olga czb   14K Jan 17 07:54 human_transcription_factors_with_uniprot_ids_random_subset100.csv
-rw-r--r--  1 olga czb  3.5K Jan 17 07:54 human_transcription_factors_with_uniprot_ids_random_subset100.csv.gz
-rw-r--r--  1 olga czb   12K Jan 17 07:54 human_transcription_factors_with_uniprot_ids_random_subset100.parquet
-rw-r--r--  1 olga czb   76K Jan 15 10:57 human_visual_transduction_with_uniprot_ids.csv
-rw-r--r--  1 olga czb   19K Jan 15 10:57 

# Read human visual transduction genes

In [6]:
visual = pd.read_csv("human_visual_transduction_with_uniprot_ids.csv")
print(visual.shape)
visual.head()

(3053, 3)


,source__uniprot_id,source__id_type,source__db_id
0,P29973,UniProtKB-ID,CNGA1_HUMAN
1,P29973,Gene_Name,CNGA1
2,P29973,Gene_Synonym,CNCG
3,P29973,Gene_Synonym,CNCG1
4,P29973,GI,2506302


## [Opisthokonta](https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?lvl=0&amp;id=33154) taxonomy ID

In [7]:
opisthokonta_taxonomy_id = '33154'

In [8]:
cd /home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019

/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019


# Use ENSEMBL Rest API to get homologous genes

### Separate out samples with ensg ids and not

In [9]:
ensg_rows = visual.source__db_id.str.startswith("ENSG0")
visual_non_ensg = visual.loc[~ensg_rows]
print(visual_non_ensg.shape)
visual_non_ensg.head()

(2981, 3)


,source__uniprot_id,source__id_type,source__db_id
0,P29973,UniProtKB-ID,CNGA1_HUMAN
1,P29973,Gene_Name,CNGA1
2,P29973,Gene_Synonym,CNCG
3,P29973,Gene_Synonym,CNCG1
4,P29973,GI,2506302


In [10]:
visual_ensg = visual.loc[ensg_rows]
print(visual_ensg.shape)
visual_ensg.head()

(72, 3)


,source__uniprot_id,source__id_type,source__db_id
42,P29973,Ensembl,ENSG00000198515
119,P16520,Ensembl,ENSG00000111664
212,P11488,Ensembl,ENSG00000114349
283,Q02846,Ensembl,ENSG00000132518
370,P35913,Ensembl,ENSG00000133256


In [11]:
from urllib.error import HTTPError

In [12]:
ensembl_id = 'ENSG00000198515'

In [14]:
response = ensembl_rest.homology_ensemblgene(ensembl_id, target_taxon=opisthokonta_taxonomy_id, cigar_line=False, sequence=None)

In [18]:
response = ensembl_rest.homology_ensemblgene(ensembl_id, target_taxon=opisthokonta_taxonomy_id, cigar_line=False, 
                                                     sequence='protein', aligned=False,
                                                     type='orthologs')

In [27]:
import requests

def get_orthologues(ensembl_id, target_taxon, verbose=False, type='orthologues', cigar_line=False, sequence='protein', aligned=False):
    server = "https://rest.ensembl.org"
    ext = f"/homology/id/{ensembl_id}?" \
        f";type={type};cigar_line={int(cigar_line)};sequence={sequence};target_taxon={target_taxon};aligned={int(aligned)}"

    r = requests.get(server + ext,
                     headers={"Content-Type": "application/json"})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    decoded = r.json()
    # print(repr(decoded))
    if verbose:
        pprint(decoded)
    return decoded
response = get_orthologues(ensembl_id, opisthokonta_taxonomy_id)

In [28]:
data = response['data']
homologies = data[0]['homologies']
homologies[0]

{'source': {'id': 'ENSG00000198515',
  'species': 'homo_sapiens',
  'seq': 'MESRSSPRLECSGAISAHCSLHLPDSSDFQLIFVFLVEMGFHHVGQAGLELLISSDLPTSASQSAGITDMKLSMKNNIINTQQSFVTMPNVIVPDIEKEIRRMENGACSSFSEDDDSASTSEESENENPHARGSFSYKSLRKGGPSQREQYLPGAIALFNVNNSSNKDQEPEEKKKKKKEKKSKSDDKNENKNDPEKKKKKKDKEKKKKEEKSKDKKEEEKKEVVVIDPSGNTYYNWLFCITLPVMYNWTMVIARACFDELQSDYLEYWLILDYVSDIVYLIDMFVRTRTGYLEQGLLVKEELKLINKYKSNLQFKLDVLSLIPTDLLYFKLGWNYPEIRLNRLLRFSRMFEFFQRTETRTNYPNIFRISNLVMYIVIIIHWNACVFYSISKAIGFGNDTWVYPDINDPEFGRLARKYVYSLYWSTLTLTTIGETPPPVRDSEYVFVVVDFLIGVLIFATIVGNIGSMISNMNAARAEFQARIDAIKQYMHFRNVSKDMEKRVIKWFDYLWTNKKTVDEKEVLKYLPDKLRAEIAINVHLDTLKKVRIFADCEAGLLVELVLKLQPQVYSPGDYICKKGDIGREMYIIKEGKLAVVADDGVTQFVVLSDGSYFGEISILNIKGSKAGNRRTANIKSIGYSDLFCLSKDDLMEALTEYPDAKTMLEEKGKQILMKDGLLDLNIANAGSDPKDLEEKVTRMEGSVDLLQTRFARILAEYESMQQKLKQRLTKVEKFLKPLIDTEFSSIEGPGAESGPIDST',
  'taxon_id': 9606,
  'perc_pos': 95.5204,
  'perc_id': 95.2569,
  'protein_id': 'ENSP00000384264'},
 'method_link_type': 'ENSEMBL_ORTHOLOGUES',
 'type': 'ortholo

In [29]:
homologies[0]['source']['protein_id']

'ENSP00000384264'

In [30]:
homologies[0]['source']['seq']

'MESRSSPRLECSGAISAHCSLHLPDSSDFQLIFVFLVEMGFHHVGQAGLELLISSDLPTSASQSAGITDMKLSMKNNIINTQQSFVTMPNVIVPDIEKEIRRMENGACSSFSEDDDSASTSEESENENPHARGSFSYKSLRKGGPSQREQYLPGAIALFNVNNSSNKDQEPEEKKKKKKEKKSKSDDKNENKNDPEKKKKKKDKEKKKKEEKSKDKKEEEKKEVVVIDPSGNTYYNWLFCITLPVMYNWTMVIARACFDELQSDYLEYWLILDYVSDIVYLIDMFVRTRTGYLEQGLLVKEELKLINKYKSNLQFKLDVLSLIPTDLLYFKLGWNYPEIRLNRLLRFSRMFEFFQRTETRTNYPNIFRISNLVMYIVIIIHWNACVFYSISKAIGFGNDTWVYPDINDPEFGRLARKYVYSLYWSTLTLTTIGETPPPVRDSEYVFVVVDFLIGVLIFATIVGNIGSMISNMNAARAEFQARIDAIKQYMHFRNVSKDMEKRVIKWFDYLWTNKKTVDEKEVLKYLPDKLRAEIAINVHLDTLKKVRIFADCEAGLLVELVLKLQPQVYSPGDYICKKGDIGREMYIIKEGKLAVVADDGVTQFVVLSDGSYFGEISILNIKGSKAGNRRTANIKSIGYSDLFCLSKDDLMEALTEYPDAKTMLEEKGKQILMKDGLLDLNIANAGSDPKDLEEKVTRMEGSVDLLQTRFARILAEYESMQQKLKQRLTKVEKFLKPLIDTEFSSIEGPGAESGPIDST'

In [ ]:

dfs = []


human_records = {}
nonhuman_records = {}

def single_homology_to_series(homology, ignore_fields=['align_seq', 'cigar_line', 'target', 'source']):

    homology_for_series = {}
    for key, value in homology.items():
        if not key in ignore_fields:
            homology_for_series[key] = value
        if key in ('target', 'source'):
            for k, v in value.items():
                if not k in ignore_fields:
                    homology_for_series[f"{key}__{k}"] = v

    series = pd.Series(homology_for_series)
    return series


for ensembl_id in tqdm(visual_ensg.source__db_id):
    sleep(1)
    try:
        response = get_orthologues(ensembl_id, opisthokonta_taxonomy_id)
    except HTTPError:
        # Probably a 503 error, meaning server is busy, so wait 2 seconds and try again
        sleep(2)
        response = get_orthologues(ensembl_id, opisthokonta_taxonomy_id)

    data = response['data']
    homologies = data[0]['homologies']
    df = pd.DataFrame(map(single_homology_to_series, homologies))
    dfs.append(df)
    for homology in homologies:
        human_id = homology['source']['protein_id']
        human_seq = homology['source']['seq']
        human_records[human_id] = human_seq

        nonhuman_id = homology['target']['protein_id']
        nonhuman_seq = homology['target']['seq']
        nonhuman_records[nonhuman_id] = nonhuman_seq
    
visual_opsithokonts_ensembl = pd.concat(dfs, ignore_index=True)
print(visual_opsithokonts_ensembl.shape)
visual_opsithokonts_ensembl.head()

In [40]:
len(human_records)

30

In [41]:
len(nonhuman_records)

7068

In [42]:
def write_fasta(records, fasta):
    with open(fasta, 'w') as f:
        for key, value in records.items():
            f.write(f">{key}\n{value}\n")

In [50]:
outdir
# ! mkdir $outdir

'/mnt/data_sm/olga/kmer-hashing/visual-transduction-ensembl/'

In [55]:
visual_opsithokonts_ensembl.head()

,dn_ds,method_link_type,source__id,source__perc_id,source__perc_pos,source__protein_id,source__seq,source__species,source__taxon_id,target__id,target__perc_id,target__perc_pos,target__protein_id,target__seq,target__species,target__taxon_id,taxonomy_level,type
0,0.35714,ENSEMBL_ORTHOLOGUES,ENSG00000198515,95.2569,95.5204,ENSP00000384264,MESRSSPRLECSGAISAHCSLHLPDSSDFQLIFVFLVEMGFHHVGQ...,homo_sapiens,9606,ENSGGOG00000013840,99.3132,99.5879,ENSGGOP00000013502,TESRSLPRLECSGAISAHCSLHLPDSSDFQLIFVFLVDMKLSMKNN...,gorilla_gorilla,9595,Homininae,ortholog_one2one
1,0.13376,ENSEMBL_ORTHOLOGUES,ENSG00000198515,88.1423,88.1423,ENSP00000384264,MESRSSPRLECSGAISAHCSLHLPDSSDFQLIFVFLVEMGFHHVGQ...,homo_sapiens,9606,ENSPTRG00000046448,99.5536,99.5536,ENSPTRP00000088384,MKNNIINTQQSFVTMPNVIVPDIEKEIRRMENGACSSFSEDDDSAS...,pan_troglodytes,9598,Homininae,ortholog_one2many
2,0.17450,ENSEMBL_ORTHOLOGUES,ENSG00000198515,99.4730,99.4730,ENSP00000384264,MESRSSPRLECSGAISAHCSLHLPDSSDFQLIFVFLVEMGFHHVGQ...,homo_sapiens,9606,ENSPPAG00000021167,99.4730,99.4730,ENSPPAP00000005085,MESRSSPRLECSGAISAHCSLHLPDSSDFQLIFVFLVEMGFHHVGQ...,pan_paniscus,9597,Homininae,ortholog_one2one
3,0.08387,ENSEMBL_ORTHOLOGUES,ENSG00000198515,99.7365,99.7365,ENSP00000384264,MESRSSPRLECSGAISAHCSLHLPDSSDFQLIFVFLVEMGFHHVGQ...,homo_sapiens,9606,ENSPTRG00000016036,99.7365,99.7365,ENSPTRP00000027592,MESRSSPRLECSGAISAHCSLHLPDSSDFQLIFVFLVEMGFHHVGQ...,pan_troglodytes,9598,Homininae,ortholog_one2many
4,0.17989,ENSEMBL_ORTHOLOGUES,ENSG00000198515,90.2503,90.5138,ENSP00000384264,MESRSSPRLECSGAISAHCSLHLPDSSDFQLIFVFLVEMGFHHVGQ...,homo_sapiens,9606,ENSPPYG00000014712,99.2754,99.5652,ENSPPYP00000016435,MKLSMKKNIINTQQSFVTMPNVIVPDIEKEIRRMENGACSSFSEDD...,pongo_abelii,9601,Hominidae,ortholog_one2one


In [57]:
visual_opsithokonts_ensembl.shape

(17013, 18)

In [59]:
visual_opsithokonts_ensembl.to_parquet(f'{outdir}/visual_transduction_protein_ensembl_orthology.parquet')
visual_opsithokonts_ensembl.to_csv(f'{outdir}/visual_transduction_protein_ensembl_orthology.csv', index=False)

In [51]:
write_fasta(human_records, f'{outdir}/human_visual_transduction_proteins_ensembl.fasta')
write_fasta(nonhuman_records, f'{outdir}/nonhuman_visual_transduction_proteins_ensembl.fasta')

In [52]:
pwd

'/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019'

In [54]:
ls -lha $outdir

total 3.1M
drwxr-xr-x 2 olga root 4.0K Jan 17 19:02 ./
drwxrwxr-x 9 1004 1004 4.0K Jan 17 19:02 ../
-rw-r--r-- 1 olga czb   15K Jan 17 19:02 human_visual_transduction_proteins_ensembl.fasta
-rw-r--r-- 1 olga czb  3.0M Jan 17 19:02 nonhuman_visual_transduction_proteins_ensembl.fasta


# Script to run

In [ ]:
%%file qfo_human_vs_opisthokont_visual_ensembl.sh
#!/bin/bash
OUTDIR=/mnt/data_sm/olga/kmer-hashing/visual-transduction-ensembl/
mkdir -p $OUTDIR/intermediates
cd $OUTDIR/intermediates

PARQUET=$OUTDIR/visual-transduction-proteins-ensembl.parquet

HUMAN=$OUTDIR/human_visual_transduction_proteins_ensembl.fasta
NOT_HUMAN=$OUTDIR/nonhuman_visual_transduction_proteins_ensembl.fasta

conda activate khtools--encodings--compare-cli


time khtools compare-kmers \
    --processes 120 \
    --ksize-min 3 \
    --ksize-max 45 \
    --parquet $PARQUET \
    --fastas2 $HUMAN \
    --no-csv \
    $NOT_HUMAN | tee khtools_compare-kmers.log